# Projeto 2 - Ciência dos Dados

**Nome: Breno Marti**

**Nome: João Pedro Chacon Ruiz**

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [116]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emojis

In [117]:
import tweepy
import math
import os.path
import pandas as pd
import json
import numpy as np
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [118]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [119]:
#Produto escolhido:
produto = 'Bugatti'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [143]:
'''
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
    if (not msg.retweeted) and ('RT' not in msg.full_text):
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs'
'''

'\n#Cria um objeto para a captura\napi = tweepy.API(auth)\n\n#Inicia a captura, para mais detalhes: ver a documentação do tweepy\ni = 1\nmsgs = []\nfor msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():\n    if (not msg.retweeted) and (\'RT\' not in msg.full_text):\n        msgs.append(msg.full_text.lower())\n        i += 1\n    if i > n:\n        break\n\n#Embaralhando as mensagens para reduzir um possível viés\nshuffle(msgs\'\n'

Salvando os dados em uma planilha Excel:

In [144]:
'''
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()
'''

"\n#Verifica se o arquivo não existe para não substituir um conjunto pronto\nif not os.path.isfile('./{0}.xlsx'.format(produto)):\n    \n    #Abre o arquivo para escrita\n    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))\n\n    #divide o conjunto de mensagens em duas planilhas\n    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})\n    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)\n\n    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})\n    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)\n\n    #fecha o arquivo\n    writer.save()\n"

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [162]:
raw = pd.read_excel('Bugatti_treino.xlsx')

In [163]:
relevante_treino_raw = raw[raw['Relevância'] == 1]

In [164]:
nao_relevante_treino_raw = raw[raw["Relevância"] == 0]

#### Funções de limpeza de texto:


In [165]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[#@,!\()-.€":?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed   

In [166]:
import emojis

In [167]:
def transform_emojis(text):
    clean = emojis.decode(text)
    return clean

#### Limpando e transformando os dataframes em séries para a análise:

In [168]:
l = ['\n','https','//t','co/']

In [169]:
list1 = []
for tweet in relevante_treino_raw['Treinamento']:
    texto = transform_emojis(tweet)
    list1.append(cleanup(texto))
    
relevante_treino = pd.Series(list1)

In [170]:
relevante_treino = relevante_treino.str.replace('|'.join(l),'',regex=True).str.strip()
relevante_treino

0     mds cade a koenigsegg pra bater esses 500km/h ...
1     bugatti bate recorde de velocidade com um carr...
2     depois de bater o próprio recorde de 490 km/h ...
3     podem não ser os mais bonitos  mas os carros d...
4     parece que teremos o bugatti chiron super spor...
5     bugatti descobriu o quão rápido o chiron é  fl...
6     bugatti chiron com velocidade final de 490 km/...
7      quase 500 kph em um carro  absurdo    cahutq6joi
8     tô pesquisando uns carros luxuosos aqui e perc...
9     após 490 km/h do chiron  bugatti anuncia que n...
10    bugatti chiron super sport 300   versão  civil...
11    bugatti ultrapassa a marca dos 480 km/h – e de...
12    bugatti quebra o recorde de velocidade com um ...
13    bugatti chiron chega a 490 km/h e bate recorde...
14    carro mais rápido do mundo  nova bugatti terá ...
15    williamenb outra coisa  porque aquele bugatti ...
16    wow  a bugatti chiron chegou a 490 km/h    iss...
17    bugatti chiron  levemente modificada  cheg

In [171]:
list2 = []
for tweet in nao_relevante_treino_raw['Treinamento']:
    texto = transform_emojis(tweet)
    list2.append(cleanup(texto))
    
nao_relevante_treino = pd.Series(list2)

In [172]:
nao_relevante_treino = nao_relevante_treino.str.replace('|'.join(l),'',regex=True).str.strip()
nao_relevante_treino

0      talla 42 eu  bugatti 3 11382e 11  botas clasic...
1      talla 40 eu  bugatti 3 11384e 11  botas clasic...
2      carolinaazered2  junior__bugatti com o bumbum ...
3                fuga de bugatti com ela depois das três
4      bugatti verón supersport motor triturbo 8 0 l ...
5      el nuevo récord de velocidad del bugatti chiro...
6      se continuar esse calor vai todo mundo derrete...
7                                 lleobertodo menino bom
8      gente vamos jogar stop  não vale olhar na inte...
9      aberto até de madrugada  bugatti chiron chega ...
10     gente vamos jogar stop  não vale olhar na inte...
11     luaracaastro desse jeito  daqui a pouco volta ...
12                    jenniferbraaga uma delicinha kkkkk
13                              paloma_souza18 foi ontem
14     ballerichh claro q  lembro alugamos um  bugatt...
15     mas eu conseguiria trazer um suco gástrico esp...
16              ofereçam me um bugatti  não peço mais xd
17     iamthebrunao tomara q nã

In [173]:
list3 = []
for tweet in raw['Treinamento']:
    texto = transform_emojis(tweet)
    list3.append(cleanup(texto))
    
tudo = pd.Series(list3)

In [174]:
tudo = tudo.str.replace('|'.join(l),'',regex=True).str.strip()
tudo

0      talla 42 eu  bugatti 3 11382e 11  botas clasic...
1      talla 40 eu  bugatti 3 11384e 11  botas clasic...
2      carolinaazered2  junior__bugatti com o bumbum ...
3      mds cade a koenigsegg pra bater esses 500km/h ...
4      bugatti bate recorde de velocidade com um carr...
5                fuga de bugatti com ela depois das três
6      bugatti verón supersport motor triturbo 8 0 l ...
7      el nuevo récord de velocidad del bugatti chiro...
8      se continuar esse calor vai todo mundo derrete...
9      depois de bater o próprio recorde de 490 km/h ...
10                                lleobertodo menino bom
11     gente vamos jogar stop  não vale olhar na inte...
12     podem não ser os mais bonitos  mas os carros d...
13     aberto até de madrugada  bugatti chiron chega ...
14     gente vamos jogar stop  não vale olhar na inte...
15     luaracaastro desse jeito  daqui a pouco volta ...
16                    jenniferbraaga uma delicinha kkkkk
17                             

#### Contando o número de aparições de cada palavra :

In [175]:
list4 = []
for tweet in tudo:
    list4 = list4 + tweet.split()

universo = pd.Series(list4)
universo.value_counts()

bugatti            223
de                 128
a                  124
o                  119
que                 75
um                  74
e                   69
do                  67
chiron              65
não                 62
na                  55
km/h                46
carro               46
nome                40
490                 40
eu                  39
é                   39
com                 37
em                  35
mais                33
primeira            33
seu                 30
letra               26
gente               26
para                26
por                 25
da                  24
velocidade          24
recorde             23
bananaobjeto        23
                  ... 
todos                1
usa                  1
garrafa              1
centodirci           1
dentro               1
gostando             1
kkkkkkkkkk           1
festa                1
lyvlzkuist           1
link                 1
faria                1
altezasemtrono       1
megane…    

In [176]:
list5 = []
for tweet in relevante_treino:
    list5 = list5 + tweet.split()

palavras_relevantes = pd.Series(list5)
palavras_relevantes.value_counts()

bugatti            76
o                  55
a                  53
de                 52
chiron             45
km/h               38
490                31
que                31
carro              27
um                 26
do                 23
em                 22
mais               20
velocidade         20
recorde            19
e                  17
com                15
é                  15
não                14
da                 13
300                11
mph                10
sport              10
os                  9
chega               8
mundial             8
super               8
500                 7
por                 7
eu                  7
                   ..
distintas           1
sobrealimentado     1
trabalho            1
cima                1
ye7je7pxvg          1
grupo               1
rny6wdw9tc          1
bonitos             1
jgz1d7afi7          1
descobrir           1
linhas              1
veyron              1
média               1
foda                1
5         

In [177]:
list6 = []
for tweet in nao_relevante_treino:
    list6 = list6 + tweet.split()

palavras_nao_relevantes = pd.Series(list6)
palavras_nao_relevantes.value_counts()

bugatti               147
de                     76
a                      71
o                      64
e                      52
na                     49
não                    48
um                     48
do                     44
que                    44
nome                   39
eu                     32
primeira               32
seu                    29
letra                  26
gente                  25
é                      24
bananaobjeto           23
vamos                  22
jogar                  22
com                    22
olhar                  22
vale                   22
stop                   21
brancofruta            21
respondanome           21
bugatticor             20
chiron                 20
internet               19
para                   19
                     ... 
quatrocentos            1
s15                     1
triunfo                 1
lanço                   1
ficar                   1
3tzw5cfyey              1
hipótese                1
jatinho     

#### Frequência relativa de cada palavra:

In [178]:
universo.value_counts(True)

bugatti            0.044511
de                 0.025549
a                  0.024750
o                  0.023752
que                0.014970
um                 0.014770
e                  0.013772
do                 0.013373
chiron             0.012974
não                0.012375
na                 0.010978
km/h               0.009182
carro              0.009182
nome               0.007984
490                0.007984
eu                 0.007784
é                  0.007784
com                0.007385
em                 0.006986
mais               0.006587
primeira           0.006587
seu                0.005988
letra              0.005190
gente              0.005190
para               0.005190
por                0.004990
da                 0.004790
velocidade         0.004790
recorde            0.004591
bananaobjeto       0.004591
                     ...   
todos              0.000200
usa                0.000200
garrafa            0.000200
centodirci         0.000200
dentro             0

In [179]:
palavras_relevantes.value_counts(True)

bugatti            0.051213
o                  0.037062
a                  0.035714
de                 0.035040
chiron             0.030323
km/h               0.025606
490                0.020889
que                0.020889
carro              0.018194
um                 0.017520
do                 0.015499
em                 0.014825
mais               0.013477
velocidade         0.013477
recorde            0.012803
e                  0.011456
com                0.010108
é                  0.010108
não                0.009434
da                 0.008760
300                0.007412
mph                0.006739
sport              0.006739
os                 0.006065
chega              0.005391
mundial            0.005391
super              0.005391
500                0.004717
por                0.004717
eu                 0.004717
                     ...   
distintas          0.000674
sobrealimentado    0.000674
trabalho           0.000674
cima               0.000674
ye7je7pxvg         0

In [180]:
palavras_nao_relevantes.value_counts(True)

bugatti               0.041690
de                    0.021554
a                     0.020136
o                     0.018151
e                     0.014748
na                    0.013897
não                   0.013613
um                    0.013613
do                    0.012479
que                   0.012479
nome                  0.011061
eu                    0.009075
primeira              0.009075
seu                   0.008225
letra                 0.007374
gente                 0.007090
é                     0.006807
bananaobjeto          0.006523
vamos                 0.006239
jogar                 0.006239
com                   0.006239
olhar                 0.006239
vale                  0.006239
stop                  0.005956
brancofruta           0.005956
respondanome          0.005956
bugatticor            0.005672
chiron                0.005672
internet              0.005389
para                  0.005389
                        ...   
quatrocentos          0.000284
s15     

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [181]:
P_relevante = palavras_relevantes.size/universo.size
P_relevante

0.2962075848303393

In [182]:
P_nao_relevante = palavras_nao_relevantes.size/universo.size
P_nao_relevante

0.7037924151696606

In [198]:
raw2 = pd.read_excel('Bugatti_teste.xlsx')
relevante_teste_raw2 = raw2[raw2['Relevância'] == 1]
nao_relevante_teste_raw2 = raw2[raw2["Relevância"] == 0]

lista1 = []
for tweet in relevante_teste_raw2['Teste']:
    texto = transform_emojis(tweet)
    lista1.append(cleanup(texto))
    
relevante_teste = pd.Series(lista1)
relevante_teste = relevante_teste.str.replace('|'.join(l),'',regex=True).str.strip()

lista2 = []
for tweet in nao_relevante_teste_raw2['Teste']:
    texto = transform_emojis(tweet)
    lista2.append(cleanup(texto))
    
nao_relevante_teste = pd.Series(lista2)
nao_relevante_teste = nao_relevante_teste.str.replace('|'.join(l),'',regex=True).str.strip()

lista3 = []
for tweet in raw2['Teste']:
    texto = transform_emojis(tweet)
    lista3.append(cleanup(texto))
    
testudo = pd.Series(lista3)
testudo = testudo.str.replace('|'.join(l),'',regex=True).str.strip()

In [199]:
lista4 = []
for tweet in testudo:
    lista4 = lista4 + tweet.split()

universo_teste = pd.Series(lista4)
universo_teste.value_counts()

bugatti           137
a                  92
de                 73
o                  66
e                  63
do                 51
que                51
chiron             47
não                43
km/h               42
um                 39
é                  37
490                32
com                31
carro              31
recorde            28
na                 26
uma                23
nome               21
eu                 21
em                 20
mais               19
seu                18
primeira           18
bananaobjeto       17
no                 17
300                17
pra                17
da                 16
se                 16
                 ... 
andy                1
o55nngb6ce          1
aoooo               1
baleno              1
música              1
989k28qply          1
1937                1
birra               1
sua                 1
8tcfzogpid          1
tonytheboy_         1
vcs                 1
sallyeaves          1
divulga             1
monegasco 

In [200]:
lista5 = []
for tweet in relevante_teste:
    lista5 = lista5 + tweet.split()

palavras_relevantes_teste = pd.Series(lista5)
palavras_relevantes_teste.value_counts()

bugatti         77
a               55
chiron          42
o               39
km/h            38
de              33
490             32
e               29
recorde         28
que             21
um              19
do              19
carro           18
é               17
velocidade      16
300             15
chega           14
em              14
com             14
mais            10
quebra          10
não             10
astonished      10
novo             9
mph              9
super            9
mundial          9
aos              8
vídeo            8
os               8
                ..
gosto            1
está             1
skrb9j6wei       1
amanhã           1
2021…            1
cusz1carqa       1
tamanho          1
vendo            1
primeira         1
j4tzlwrcau       1
hehe             1
htmkvujwpi       1
respeito         1
venon            1
onde             1
torna            1
zwbpoqx43l       1
testou           1
éeste            1
batia            1
ijyqk0wq74       1
insano      

In [201]:
lista6 = []
for tweet in nao_relevante_teste:
    lista6 = lista6 + tweet.split()

palavras_nao_relevantes_teste = pd.Series(lista6)
palavras_nao_relevantes_teste.value_counts()

bugatti             60
de                  40
a                   37
e                   34
não                 33
do                  32
que                 30
o                   27
na                  24
é                   20
uma                 20
um                  20
nome                20
com                 17
eu                  17
bananaobjeto        17
primeira            17
bugatticor          16
seu                 16
pra                 15
letra               15
vamos               15
gente               14
stop                14
jogar               14
respondanome        14
vale                14
olhar               14
brancofruta         13
no                  13
                    ..
favor                1
desapareçam          1
19                   1
putona               1
brinca               1
vermelhobebida       1
mini                 1
benz                 1
muit                 1
dar                  1
indo                 1
j1cnebqdis           1
“bugatti”  

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**